In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
from scipy.interpolate import make_interp_spline

In [7]:
INTERESTED_FEATURE = 'Pupil X'
GRAPH_SUBJECT_DATA = True 

In [4]:
df = pd.read_csv('DataFrames/' + INTERESTED_FEATURE +'_Response_Data')

In [5]:
def getEyeSpeedInfo(y_points, x_points, windowTime):
        # now that we have the average graph, take derivative of graph to get speed 
        dy = np.diff(y_points)
        dx = np.diff(x_points)
        
        speed_y = dy / dx
        adjust_x = np.linspace(0, windowTime, len(y_points) - 1) # since we cant take derivative of last point 
        
        return adjust_x, speed_y



In [15]:


def graphSubject(subject_id, task, data):

    # [Get Data and Features]
    time, speed, pupil_data, avg_diameter_points, std_diameter_points, x, norm  = data
    
    # [Plot Data]
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10, 7))
    fig.tight_layout(pad=3.0)
    fig.delaxes(ax4)
        
    # plot the avg response 
    ax1.set_title("Subject {} Average Data For Task - {}".format(subject_id, task))
    ax1.set_xlabel("Time (s)")
    ax1.set_ylabel("Pupil Diameter (mm)")
    ax1.plot(x, avg_diameter_points, color ="b")

    # plot std dev
    ax1.plot(x, avg_diameter_points + std_diameter_points, linestyle='dashed', color='r')
    ax1.plot(x, avg_diameter_points - std_diameter_points, linestyle='dashed', color='r')

    # plot pupil speed data 
    ax2.set_title("Subject {} Eye Speed Data For Task - {}".format(subject_id, task))
    ax2.set_xlabel("Time (s)")
    ax2.set_ylabel("Pupil Speed (mm)")
    ax2.plot(time, speed, color ="b")
    
    ax2.plot(x, avg_diameter_points + std_diameter_points, linestyle='dashed', color='r')
    ax2.plot(x, avg_diameter_points - std_diameter_points, linestyle='dashed', color='r')
    
    
    # plot smooth speed data        
    ax3.set_title(" Subject {} Normalized Data For Task - {}".format(subject_id, task))
    ax3.set_xlabel("Time (s)")
    ax3.set_ylabel("Pupil Speed (mm)")
    ax3.plot(x, norm, color ="b")
    
    if GRAPH_SUBJECT_DATA:
        plt.show()
        
    # save plot  
    plt.savefig("Average_Data_Graphs/"+ task+"_Graphs/Subject_"+ str(subject_id) + "_" + task)
    plt.clf()
    plt.close()

def getPupilData(df, task):
    pupil_data_holder = [] 
    normalized_pupil_data = []
    speed_data  = []
    
    for subject_id in df['SID'].unique():

        condition = ((df['Task Type'] == task) & (df['SID'] == subject_id))
        target_data = df[condition]
        
        
        # gather all the relevant pupil data 
        print("Gathering Data For Subject " + str(subject_id))
        
        # These will be our trackers for the current subject 
        curr_sub_data = []
        curr_norm_data = []
            
        for block in target_data['Block'].unique():
            
            # define condition and get values 
            condition2 = target_data['Block'] == block 
            block_data = target_data[condition2]
                     
            # go through all the pupil data rows, and convert them into a float list (for each block, there shouldd be a certain amount of stimulus onset times presented: ex for DPT there are 160 stimulus shown)
            for data in block_data[INTERESTED_FEATURE]:
                print(data)
                # convert data represented as string to a list 
                converted = literal_eval(data)
                
                # skip over missing data in time series 
                if len(converted) == 0:
                    continue
                
                curr_sub_data.append(converted) # holds current subjects data 
                pupil_data_holder.append(converted) # holds every subjects data
            
            if len(curr_sub_data) == 0:
                # missing block data (ommit)
                print('x')
            else:
                # got data for block 
                print('o ', end=' ')
            
        # normalize the data by subtracting out the average for the current time series we are looking at 
        flatten_list = [j for sub in curr_sub_data for j in sub] # convert 2d list to 1d so we can avg 
        avg_value = sum(flatten_list) / len(flatten_list)
            
        curr_norm_data = [[(x - avg_value) for x in row] for row in curr_sub_data]
        normalized_pupil_data.extend(curr_norm_data)
            
        # get different features to be graphed 
        curr_pupil_data = np.array(curr_sub_data)
        # calculate avg pupil size
        curr_avg_diameter_points = np.average(curr_pupil_data, axis=0)
        curr_std_diameter_points = np.std(curr_pupil_data, axis=0)   
        curr_x = np.linspace(0, window_time[task], len(curr_avg_diameter_points))
        
        # calculate speed
        curr_time, curr_speed = getEyeSpeedInfo(curr_avg_diameter_points, curr_x, window_time[task])
        # calculate normalized average 
        curr_norm = np.array(curr_norm_data)
        curr_norm_avg = np.average(curr_norm, axis= 0)
     
        # call graph function and give it data 
        data = [curr_time, curr_speed, curr_pupil_data, curr_avg_diameter_points, curr_std_diameter_points, curr_x, curr_norm_avg]
        graphSubject(subject_id, task, data)
            
        # add speed to the list tracking all subject speeds 
        speed_data.append(curr_speed.tolist())
        print("Done")
    
    return pupil_data_holder, normalized_pupil_data, speed_data

In [16]:


window_time = {'DPT': 3, 'MA': 14, 'PVT': 3, 'VWM': 3}

tasks = df['Task Type'].unique()

for task in tasks:
    print("Task:",task)

    pupil_data_holder, normalized_pupil_data, speed_data = getPupilData(df, task)
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10, 7))
    fig.tight_layout(pad=3.0)
    fig.delaxes(ax4)
    
    # [Plot the Average Pupil Size]
    final = np.array(pupil_data_holder)
    final_avg = np.average(final, axis=0)    
    final_std = np.std(final, axis=0)
    x = np.linspace(0, window_time[task], len(final_avg))
    
    ax1.set_title("Average Pupil Response For Task " + task)
    ax1.set_xlabel("Time (s)")
    ax1.set_ylabel("Pupil Diameter (mm)")
    ax1.plot(x, final_avg, color ="b")

    # plot std dev
    ax1.plot(x, final_avg + final_std, linestyle='dashed', color='r')
    ax1.plot(x, final_avg - final_std, linestyle='dashed', color='r')
    
    
    # [Plot the Average Speed]  
    final = np.array(speed_data)
    final_avg = np.average(final, axis=0)    
    final_std = np.std(final, axis=0)
    x = np.linspace(0, window_time[task], len(final_avg))
    
    # set labels and plot 
    ax2.set_title("Average Pupil Speed For Task " + task)
    ax2.set_xlabel("Time (s)")
    ax2.set_ylabel("Pupil Diameter (mm/s)")
    ax2.plot(x, final_avg, color ="b")

    # plot std dev
    ax2.plot(x, final_avg + final_std, linestyle='dashed', color='r')
    ax2.plot(x, final_avg - final_std, linestyle='dashed', color='r')
    
    
    # [Plot the normalized Average Pupil Size]  
    final = np.array(normalized_pupil_data)
    final_avg = np.average(final, axis=0)    
    final_std = np.std(final, axis=0)
    x = np.linspace(0, window_time[task], len(final_avg))
    
    # set labels and plot 
    ax3.set_title("Normalized Pupil Size For Task " + task)
    ax3.set_xlabel("Time (s)")
    ax3.set_ylabel("Pupil Diameter (mm)")
    ax3.plot(x, final_avg, color ="b")

    # plot std dev
    ax3.plot(x, final_avg + final_std, linestyle='dashed', color='r')
    ax3.plot(x, final_avg - final_std, linestyle='dashed', color='r')
    
    
    #plt.show()
    plt.savefig("Average_Data_Graphs/"+task+"_Graphs/"+task+"_All_Subject_Pupil_Data_Average")
    plt.clf()
    plt.close()

Task: DPT
Gathering Data For Subject 0
[550.2000122070311, 550.5999755859375, 550.4000244140625, 550.0999755859375, 550.0, 550.0, 550.2999877929689, 550.5999755859375, 550.7000122070311, 550.5, 550.4000244140625, 550.7999877929689, 550.4000244140625, 550.0, 549.5999755859375, 550.2999877929689, 550.5999755859375, 551.0, 551.0, 550.7999877929689, 550.5999755859375, 550.2999877929689, 550.5, 551.2000122070311, 551.0999755859375, 551.0999755859375, 550.9000244140625, 550.9000244140625, 550.7999877929689, 551.2000122070311, 551.5999755859375, 551.4000244140625, 551.2999877929689, 551.2999877929689, 551.5, 551.4000244140625, 551.2000122070311, 552.2000122070311, 551.9000244140625, 550.7999877929689, 550.7000122070311, 550.5999755859375, 550.7000122070311, 550.7999877929689, 550.7000122070311, 550.0, 550.2999877929689, 550.4000244140625, 549.9000244140625, 550.2999877929689, 551.0999755859375, 551.7000122070311, 551.5, 551.2000122070311, 551.2999877929689, 551.2000122070311, 550.700012207031

ValueError: malformed node or string: <_ast.Name object at 0x2aaaddd09430>

In [13]:
print(dpt_df.keys())

Index(['SID', 'Task Type', 'Block', 'Stimulus Index', 'Pupil Data'], dtype='object')
